In [ ]:
# for each ligand, generates ligand descriptor & coordinates of the voxels in the descriptor
# each pair of data (descriptor, centers) is saved as a pickle file

# outputs a csv containing ligand id & pickle path

In [5]:
# generate ligand descriptor files + csv 
from htmd.ui import *
from htmd.molecule.voxeldescriptors import *
from pathlib import Path
import numpy as np
import pickle as pk
from multiprocessing import Pool
import pandas as pd
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
ligands = []
ligand_root = './processed_data/ligands'
Path(ligand_root).mkdir(exist_ok = True, parents=True)

In [3]:
raw_ligand_paths = sorted(list(Path('./training_data').glob('*lig_cg.pdb')))

In [6]:
dests = []
for idx, lig_path in tqdm(enumerate(raw_ligand_paths)):
    mol = Molecule(str(lig_path), keepaltloc='all')
    bb = htmd.molecule.util.boundingBox(mol)

    xx = (bb[1][0] + bb[0][0])/2 - 12
    yy = (bb[1][1] + bb[0][1])/2 - 12
    zz = (bb[1][2] + bb[0][2])/2 - 12

    centers = []
    for ix in range(24):
        for iy in range(24):
            for iz in range(24):
                centers.append([xx + ix, yy + iy, zz + iz])

    features, centers = getVoxelDescriptors(mol, usercenters=np.array(centers), voxelsize=1, method='NUMBA')
    features = features.reshape(24, 24, 24, features.shape[1])
    
    dest = ligand_root + '/' + str(idx+1).zfill(4) + '.pk'
    with open(dest, 'wb') as f:
        pk.dump((features, centers), f)
        dests.append(dest)
        #print('done with '+str(idx+1))

0it [00:00, ?it/s]/home/alex/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:200: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(mol._dtypes[field], np.float) \
3000it [04:29, 11.15it/s]


In [7]:
idxs = list(range(1,3001))
df = pd.DataFrame({'id':idxs, 'path': dests})
df.to_csv('ligand_data.csv', index=None)

In [ ]:
'''
for idx, lig_path in enumerate(raw_ligand_paths[872:]):
    
    mol = Molecule(str(lig_path), keepaltloc='all')
    features, centers, N = getVoxelDescriptors(mol, usercenters=None, voxelsize=1, method='NUMBA')
    
    # reshape features into 3D grid x num features
    features = features.reshape(N[0], N[1], N[2], features.shape[1])
    
    # calculate padding to apply on features to get 24x24x24x8
    # also make each dim an even number, for ease in counting
    x_low_pad = 0
    x_high_pad = 0
    y_low_pad = 0
    y_high_pad = 0
    z_low_pad = 0
    z_high_pad = 0
    
    x, y, z, _ = features.shape
    
    if x % 2 == 1:
        x_low_pad +=1
        x += 1
    if y % 2 == 1:
        y_low_pad +=1
        y += 1
    if z % 2 == 1:
        z_low_pad +=1
        z += 1
    
    if x < 24:
        shift_by = (24 - x) / 2
        x_low_pad += shift_by
        x_high_pad += shift_by
        
    if y < 24:
        shift_by = (24 - y) / 2
        y_low_pad += shift_by
        y_high_pad += shift_by
        
    if z < 24:
        shift_by = (24 - z) / 2
        z_low_pad += shift_by
        z_high_pad += shift_by
    
    new_features = np.pad(features, ((int(x_low_pad), int(x_high_pad)), \
                                      (int(y_low_pad), int(y_high_pad)), \
                                      (int(z_low_pad), int(z_high_pad)),\
                                         (0,0)),
                          'constant', constant_values=0)
    
    # find which features to grab (some dimensions may be > 24)
    xp, yp, zp, f = new_features.shape
    x_low = 0; x_high = xp
    y_low = 0; y_high = yp
    z_low = 0; z_high = zp
    
    x_shift_by = 0
    if xp > 24:
        x_shift_by = (xp - 24)/2
        x_low += x_shift_by
        x_high -= x_shift_by
        
    y_shift_by = 0
    if yp > 24:
        y_shift_by = (yp - 24)/2
        y_low += y_shift_by
        y_high -= y_shift_by
        
    z_shift_by = 0
    if zp > 24:
        z_shift_by = (zp - 24)/2
        z_low += z_shift_by
        z_high -= z_shift_by
        
    # crop 24x24x24x8 from features
    cropped_features = new_features[int(x_low):int(x_high), 
                                    int(y_low):int(y_high),
                                    int(z_low):int(z_high),
                                    :]
    
    
    #if (x_high != 24):
    #    pdb.set_trace()
    
    # generate corresponding centers, for ease in preprocessing proteins
    xc_low, yc_low, zc_low = centers[0]
    xc_low -= x_low_pad + x_shift_by
    yc_low -= y_low_pad + y_shift_by
    zc_low -= z_low_pad + z_shift_by
    
    centers = []
    for ix in range(24):
        for iy in range(24):
            for iz in range(24):
                centers.append((xc_low + ix, yc_low + iy, zc_low + iz))
    
    feature_with_centers = (cropped_features, centers)
    
    dest = ligand_root + '/' + str(idx).zfill(4) + '.pk'
    with open(dest, 'wb') as f:
        pk.dump(feature_with_centers, f)
        dests.append(dest)
        print('done with '+str(idx))
'''
print('')